# QF605 Part IV

###### Prepared By:  

Dani Pangestu  
Gabriel Woon  
Gabriel Tan  
Kenneth Chong  
Peter Chettiar  
Yong Wen

### Import Necessary Packages

In [1]:
import pandas as pd
import numpy as np
from scipy import integrate
from math import log, sqrt, exp
from scipy.stats import norm


### Import Dataset

In [2]:
data_DF = pd.read_excel('df_comb.xlsx')
data_FS = pd.read_excel('df_comb.xlsx', sheet_name = "forward swap rates", header = 0, index_col = 0) 

In [3]:
data_Alpha = pd.read_excel('df_comb.xlsx', sheet_name = "alpha", header = 0, index_col = 0) 
data_Rho = pd.read_excel('df_comb.xlsx', sheet_name = "rho", header = 0, index_col = 0) 
data_Nu = pd.read_excel('df_comb.xlsx', sheet_name = "nu", header = 0, index_col = 0) 


### Define Function

In [4]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def Black76Lognormal(F, K, r, sigma, T, opt):
    d1 = (log(F/K)+(sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    if opt == 'Call':
        return F*exp(-r*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
    elif opt == 'Put':
        return K*exp(-r*T)*norm.cdf(-d2) - F*exp(-r*T)*norm.cdf(-d1)

### IRR

In [5]:
def IRR_0(K, m, N):
    # IRR(K) function
    IRR = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return IRR

def IRR_1(K, m, N):
    # 1st Derivative of IRR(K)
    first = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return first

def IRR_2(K, m, N):
    # 2nd Derivative of IRR(K)
    second = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return second

### Static Replication

In [6]:
def g_0(F):
    return F**0.25-0.2
def g_1(F):
    return 0.25*(F**(-0.75))
def g_2(F):
    return (-3/16)*(F**(-7/4))

def h_0(K, m, N):
    # implementation of h(K)
    h = g_0(K) / IRR_0(K, m, N)
    return h

def h_1(K, m, N):
    # 1st Derivative of h(K)
    first = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return first

def h_2(K, m, N):
    # 2nd Derivative of h(K)
    second = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return second

In [7]:
FSR = data_FS.iloc[9, 1]  # FSR 5y x 10y
D = data_DF.iloc[9,2]     # D(0, 5y)

alpha = data_Alpha.iloc[1,4] # alpha for 5y x 10y forward swap
beta = 0.9                        # pre-determined beta 
rho = data_Rho.iloc[1,4]      # rho for 5y x 10y forward swap
nu = data_Nu.iloc[1,4]      # nu for 5y x 10y forward swap

Tenor = 10
Delta = 0.5
T = 5

V_rec = integrate.quad(lambda x: h_2(x,Tenor,Delta)*Black76Lognormal(FSR, x, 0, SABR(FSR, x, T, alpha, 0.9, rho, nu),T, "Put"),
                                                    0,FSR)
V_pay = integrate.quad(lambda x: h_2(x,Tenor,Delta)*Black76Lognormal(FSR, x, 0, SABR(FSR, x, T, alpha, 0.9, rho, nu),T, "Call"),
                                                   FSR,100000000)
                                                   
PVoption = D * g_0(FSR) + V_rec[0] + V_pay[0]

C:\Users\Asus\AppData\Local\Temp/ipykernel_120/2965046875.py:13: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  V_rec = integrate.quad(lambda x: h_2(x,Tenor,Delta)*Black76Lognormal(FSR, x, 0, SABR(FSR, x, T, alpha, 0.9, rho, nu),T, "Put"),


In [8]:
print("PV for the Payoff is ",PVoption)

PV for the Payoff is  0.17733796755786232


### Question 2

In [9]:
Tenor = 10
Delta = 0.5
T = 5
L = 0.2**4

term1 = h_1(L, Tenor, Delta)*Black76Lognormal(FSR, L, 0, SABR(FSR, L, T, alpha, 0.9, rho, nu),T,"Call")
term2 = integrate.quad(lambda x: h_2(x, Tenor,Delta)*Black76Lognormal(FSR, x, 0, SABR(FSR, x, T, alpha, 0.9, rho, nu),T,"Call"),
                                                    L,10000000)
PV_caplet = term1+ term2[0]
print("PV of this Payoff is ",PV_caplet)

PV of this Payoff is  2.6351730460756175


C:\Users\Asus\AppData\Local\Temp/ipykernel_120/707181444.py:7: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  term2 = integrate.quad(lambda x: h_2(x, Tenor,Delta)*Black76Lognormal(FSR, x, 0, SABR(FSR, x, T, alpha, 0.9, rho, nu),T,"Call"),
